### 1. 모듈 설치

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, NoSuchFrameException, WebDriverException, UnexpectedAlertPresentException
import requests
from urllib.request import urlopen

import os
import sys
import urllib.request
import urllib.parse
import json
import re
import time

#### 네이버 API
- 60dTEJyX2yyoj4KzYXt8
- F3gbSu096f

### 2. 데이터 불러오기

In [3]:
data = pd.read_csv('./data/대전.csv')
data.head(3)

,Unnamed: 0,title,link,description,bloggername,bloggerlink,postdate
0,0,대전 <b>한밭수목원</b>(서원),https://blog.naver.com/risoha/223389586693,천리포수목원에서 처음 봤었는데 목련중에선 조금 늦게 개화하는 편이라고 나중에 확인하...,"안과 밖 여행, 흔적",blog.naver.com/risoha,20240320
1,1,<b>한밭수목원</b> 나들이,https://blog.naver.com/nodak59/223367418361,워낙 수목원 등을 좋아해서 아이들보다 제가 더 신나한것 같기도 하지만요. <b>한밭...,노닥창고,blog.naver.com/nodak59,20240228
2,2,<b>한밭수목원</b> 대전곤충생태관 / 아이랑갈만한곳,https://blog.naver.com/dorablewa/223358862414,어제 포스팅에 이어 대전에서 들렀던 <b>한밭수목원</b> 곤충생태관 포스팅!! 대...,day by day,blog.naver.com/dorablewa,20240219


In [4]:
# 블로그 정보 데이터프레임에서 link 열값 추출하기 - 개수 확인 (3750개)
len(data.link)

3750

In [5]:
# links 변수에 위에서 추출한 link 값 저장
links = [link for link in data.link]
len(links)

3750

In [ ]:
#중간에 중단된 경우
links = [link for link in data.link[851:2880]]
len(links)

### 3. 크롤링

In [6]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(3)

In [7]:
#블로그 링크 하나씩 불러서 크롤링
contents = []
for i in links:
    #블로그 링크 하나씩 불러오기
    try: driver.get(i)
    except WebDriverException:
        contents.append('')
        continue
    time.sleep(1)
    try:
        #블로그 안 본문이 있는 iframe에 접근하기
        driver.switch_to.frame("mainFrame")
    except (NoSuchFrameException, UnexpectedAlertPresentException):
        contents.append('')
        continue
    #본문 내용 크롤링하기
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        contents.append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        #a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        contents.append('')
    
driver.quit() #창닫기

In [8]:
# 중간에 오류가 나서 실행이 멈추면 contents에 오류발생전까지 내용들 들어와있는지 확인해보기
len(contents)

3750

In [9]:
# 오류 발생전까지 받아온 contents들 먼저 데이터프레임으로 바꾸고 저장해두기!
a = pd.DataFrame({'Texts': contents})
a

,Texts
0,일요일에 다시한번 대전 한밭수목원을 찾았다\n지난번 동원일부만 보고와서 이번엔 서원...
1,한밭수목원\n대전광역시 서구 둔산대로 169\n이제 곧 3월입니다.\n날씨는 영상과...
2,어제 포스팅에 이어 대전에서 들렀던\n한밭수목원 곤충생태관 포스팅!!\n대전 곤충생...
3,안녕하세요?! 보링보링입니다. ^^*\n얼마 전 낭군과 함께 두 아이를 데리고\n대...
4,[4월 소식] #대전서구 #대전서구소식 #4월소식\n서구로 떠나는 모험\n대전시민의...
...,...
3745,향적산 치유의숲 가벼운 트레킹으로 생태숲을 즐기다!\n◆ 태조 이성계가 국사를 논했...
3746,충청남도 서해안과 전라도 서해안은 초겨울 다설지로 유명하다. 겨울철 차가운 북서풍이...
3747,서천여행 서천 치유의숲에서의 힐링\n요즘 많은 분이 산을 찾으면서 자신의 건강을 돌...
3748,제주도 3주 살기\n제주도 유모차 가능\n제주도 아기랑 가볼만한 곳\n3일차\n서귀...


In [20]:
# 개별 텍스트 잘 크롤링되었는지 확인
a.Texts.iloc[-1]

'#아홉산 산행\n#회동동 버스종점~#부산치유의 숲\n2024. 1. 20(토)\n#부산등산클럽\n매주 셋째주 토요일 산행, 누구라도 환영합니다. 편안하게 오셔요, 회비 2만원\n가다가~먹다가~쉬다가~놀다가~구경하다가~농담하다가~걷다가 갑니다~\n☎ 010-5543-2277\n잠을 거의 못잤지만,\n#TRGO\n운동하기 전에 에너지 UP!\n나의 애용품, 힘이 절대로 안딸리고요 오히려 넘처나요~~운동선수에게 추천! 한 번만 먹어보면 알 수 있어요~~정말인데 사람들이 믿지를 않아서 이야기 안함~ㅋ\n#파마넥스\n09:55 회동동 버스종점 이곳에서 출발!\n10:07분 도착한\n이곳이 아홉산 산행입구가 아닙니다\n언덕을 넘어가야 해요\n10:13에 도착한 이곳이 등산로 입구입니다\n첫번째 철탑에서 언덕을 하나 넘어가면 이곳이 나옵니다.\n로프를 타야 하네요\n길을 새롭게 만들면서 정리가 된 것 같아ㅛ\n철탑에서 차 한잔 하고 출발했더니\n하양봉 전망대 도착하니\n11:00\n이 표지가 계속 있어요\n00:00\n00:22\n\n아홉산 우중의 능선길\n00:00\n00:00\n\n아홉산 우중의 산향\n비바람이 장난이 아니네요~~\n집에 있으면 뭐하노\n우중의 운치를 즐겨야지~~ㅎ\n11:48\n여기가 정상이 아닙니다요 더 가야해요\n길따라 더 앞으로 Go!\n12:00\n점심이나 먹고 가자~~\n산중에 따뜻한 진한 맛의 하동댁 재첩국이라~~\n정말 맛있시유~~\n12:44 드뎌 하홉산 정상도착!\n다시 백하여 정상을 오르기 직전에 있는 삼거리길에서 부산치유의숲 방향으로 하산 결정하여\n01:19 하산중에 한 컷!\n장년산으로 하산할 것인가?\n아니면 치유의 숲 입구로 하산할 것인가?\n비바람이 점 점 심해져서 치유의 숲입구로 하산\n01:35 부산치유의 숲 입구 도착\n버스가 없다\n콜 택시 불렀는데 운좋게 택시 기사님이 실수로 콜을 했다고 한다.\n장전역까지 택시비 만원이지만 고마움에 추가 5천원을 더 주었다\n14:40 장전역앞 커피숍에서 하잔 마무리'

In [21]:
# 파일 csv로 저장
a.to_csv('대전.csv', index=False, encoding='utf-8')

### 4. 크롤링이 중단되어 여러번 나눠질 경우 CONCAT

In [ ]:
a.to_csv('대구1차.csv', index=False, encoding='utf-8')

In [ ]:
a.to_csv('대구2차.csv', index=False, encoding='utf-8')

In [ ]:
a.to_csv('대구3차.csv', index=False, encoding='utf-8')

In [ ]:
df1 = pd.read_csv('data/대구1차.csv')
df2 = pd.read_csv('data/대구2차.csv')
df3 = pd.read_csv('data/대구3차.csv')

In [ ]:
#데이터 세개 합치기 Concat
combine = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)
combine

In [ ]:
# 본문 추출 확인 (첫번째만)
combine.Texts[0]

### 5. Spots 컬럼 추가

In [22]:
#75개 지역 추출
df = pd.read_csv('data/관광지_리뷰수.csv')
list = [li for li in df[df['Region']=='대전']['Spots']]
len(list)

75

In [23]:
#75개 50개씩 반복
spots = []
for li in list:
    for i in range(50):
        spots.append(li)
len(spots)

3750

In [24]:
# dataframe에 spots 컬럼 추가
combine = pd.read_csv('data/최종_대전.csv')
combine

,Texts
0,일요일에 다시한번 대전 한밭수목원을 찾았다\n지난번 동원일부만 보고와서 이번엔 서원...
1,한밭수목원\n대전광역시 서구 둔산대로 169\n이제 곧 3월입니다.\n날씨는 영상과...
2,어제 포스팅에 이어 대전에서 들렀던\n한밭수목원 곤충생태관 포스팅!!\n대전 곤충생...
3,안녕하세요?! 보링보링입니다. ^^*\n얼마 전 낭군과 함께 두 아이를 데리고\n대...
4,[4월 소식] #대전서구 #대전서구소식 #4월소식\n서구로 떠나는 모험\n대전시민의...
...,...
3745,향적산 치유의숲 가벼운 트레킹으로 생태숲을 즐기다!\n◆ 태조 이성계가 국사를 논했...
3746,충청남도 서해안과 전라도 서해안은 초겨울 다설지로 유명하다. 겨울철 차가운 북서풍이...
3747,서천여행 서천 치유의숲에서의 힐링\n요즘 많은 분이 산을 찾으면서 자신의 건강을 돌...
3748,제주도 3주 살기\n제주도 유모차 가능\n제주도 아기랑 가볼만한 곳\n3일차\n서귀...


### 6. 최종 파일 저장

In [26]:
#최종 파일 저장
combine.to_csv('최종_대전.csv', index=False, encoding='utf-8')

In [27]:
#파일 확인
check = pd.read_csv('data/최종_대전.csv')
check

,Texts,Spots
0,일요일에 다시한번 대전 한밭수목원을 찾았다\n지난번 동원일부만 보고와서 이번엔 서원...,한밭수목원
1,한밭수목원\n대전광역시 서구 둔산대로 169\n이제 곧 3월입니다.\n날씨는 영상과...,한밭수목원
2,어제 포스팅에 이어 대전에서 들렀던\n한밭수목원 곤충생태관 포스팅!!\n대전 곤충생...,한밭수목원
3,안녕하세요?! 보링보링입니다. ^^*\n얼마 전 낭군과 함께 두 아이를 데리고\n대...,한밭수목원
4,[4월 소식] #대전서구 #대전서구소식 #4월소식\n서구로 떠나는 모험\n대전시민의...,한밭수목원
...,...,...
3745,향적산 치유의숲 가벼운 트레킹으로 생태숲을 즐기다!\n◆ 태조 이성계가 국사를 논했...,치유의숲
3746,충청남도 서해안과 전라도 서해안은 초겨울 다설지로 유명하다. 겨울철 차가운 북서풍이...,치유의숲
3747,서천여행 서천 치유의숲에서의 힐링\n요즘 많은 분이 산을 찾으면서 자신의 건강을 돌...,치유의숲
3748,제주도 3주 살기\n제주도 유모차 가능\n제주도 아기랑 가볼만한 곳\n3일차\n서귀...,치유의숲
